In [39]:
%run imports
from src.functions import *
from src.classes import *
from src.plots import *
#195633320791780608, 195633325090480896



In [40]:
trace = Traceback(195633325090480896, 195633320791780608)
trace.create_input_file()

Input file created at giessler_traceback/two_trace/input.tsv


/usr/lib/python3/dist-packages/numpy/ma/core.py:467: RuntimeWarning:

invalid value encountered in cast



In [41]:
# Usage example
sample_config = 'sample_trace.conf'
config = SimulationConfig(sample_config)

# Change parameters with attributes
config.orbits = 100_000  # This updates the "Orbits" key in the [Simulation] section
config.steps = 1000      # This updates the "Steps" key
config.stepsize = -100
config.width = 100
config.star1 = '"two_trace/input.tsv#2"'
config.star2 = '"two_trace/input.tsv#3"'
config.outfile = f'"two_trace/trace/{round(time.time())}"'
# Save the changes
config.save_config('giessler_traceback/two_trace/trace.conf')


In [42]:
# Example: Executing the 'ls' command
execute_command('cd giessler_traceback ; examples/trace/traceback two_trace/trace.conf')


Command executed successfully:
two_trace/trace/1726707727.002+003: starting 16 threads for 100,000 'Potential' orbits:

Thread #01: finished 5,896 orbits.
Thread #15: finished 6,496 orbits.
Thread #12: finished 6,473 orbits.
Thread #10: finished 6,572 orbits.
Thread #06: finished 5,785 orbits.
Thread #13: finished 6,662 orbits.
Thread #05: finished 6,505 orbits.
Thread #09: finished 6,560 orbits.
Thread #00: finished 6,258 orbits.
Thread #04: finished 6,005 orbits.
Thread #14: finished 6,606 orbits.
Thread #07: finished 5,812 orbits.
Thread #02: finished 6,208 orbits.
Thread #11: finished 6,007 orbits.
Thread #03: finished 5,989 orbits.
Thread #08: finished 6,166 orbits.

In 117 of 100,000 orbits the stars get closer than 1 pc to each other.
The minimum separation of all orbits was 0.0665512 pc 10,700 years ago.
The mean minimum separation vector of the two stars (X,Y,Z) was:
(0.04 ± 0.52, 0.04 ± 0.13, -0.04 ± 0.04) pc.
The mean minimum separation was 0.0721726 pc (0.011 ± 0.001) Myr a

In [43]:
import os
import glob

def get_last_created_file(directory):
    # Get list of all files in the directory
    files = glob.glob(os.path.join(directory, "*"))
    
    # Ensure the directory is not empty
    if not files:
        return None
    
    # Get the file with the latest creation time
    latest_file = max(files, key=os.path.getctime)
    
    return latest_file

# Set the folder path
matches_file = "giessler_traceback/two_trace/trace/"
matches_file = get_last_created_file(matches_file)

best_params_table = best_params(matches_file)
original_input = Table.read('giessler_traceback/two_trace/input.tsv', format='ascii.tab')[:2]
best_params_table['RA'] = original_input['RA']
best_params_table['DE'] = original_input['DE']


In [44]:
temp2 = vstack([original_input,best_params_table])
temp2.write('giessler_traceback/two_trace/input.tsv', format='ascii.tab', overwrite=True)

In [45]:
# Usage example
sample_config = 'sample_trace.conf'
config = SimulationConfig(sample_config)

# Change parameters with attributes
config.orbits = 100_000  # This updates the "Orbits" key in the [Simulation] section
config.steps = 1000      # This updates the "Steps" key
config.stepsize = -100
config.width = 100
config.star1 = '"two_trace/input.tsv#4"'
config.star2 = '"two_trace/input.tsv#5"'
config.outfile = f'"two_trace/traj/{round(time.time())}"'
# Save the changes
config.save_config('giessler_traceback/two_trace/traj.conf')

In [46]:
execute_command('cd giessler_traceback ; examples/trajectories/trajectories two_trace/traj.conf')


Command executed successfully:



In [47]:
# Set the folder path
traj_file = "giessler_traceback/two_trace/traj/"
traj_file = get_last_created_file(traj_file)
table = read_trajectory_out(traj_file)
table

time,l_1,b_1,dist_1,l_2,b_2,dist_2,sep3d
yr,deg,deg,pc,deg,deg,pc,pc
int64,float64,float64,float64,float64,float64,float64,float64
0,166.11101,4.28173,2932.29335,166.11239,4.27749,2932.37933,0.2439
-100,166.11076,4.28174,2932.29335,166.11213,4.27753,2932.37933,0.242
-200,166.11052,4.28174,2932.29335,166.11186,4.27757,2932.37933,0.2402
-300,166.11028,4.28175,2932.29335,166.1116,4.27761,2932.37933,0.2383
-400,166.11004,4.28176,2932.29335,166.11133,4.27765,2932.37933,0.2364
-500,166.1098,4.28176,2932.29335,166.11107,4.27769,2932.37933,0.2346
-600,166.10955,4.28177,2932.29335,166.11081,4.27773,2932.37933,0.2327
-700,166.10931,4.28178,2932.29335,166.11054,4.27777,2932.37933,0.2309


In [48]:
from astropy.coordinates import Galactocentric
table['SkyCoord1_galactic'] = SkyCoord(l=table['l_1'],
                             b=table['b_1'],
                             distance=table['dist_1'],
                             frame='galactic'
                             )
table['SkyCoord2_galactic'] = SkyCoord(l=table['l_2'],
                             b=table['b_2'],
                             distance=table['dist_2'],
                             frame='galactic'
                             )
table['SkyCoord1_galactocentric'] = table['SkyCoord1_galactic'].transform_to(Galactocentric)
table['SkyCoord2_galactocentric'] = table['SkyCoord2_galactic'].transform_to(Galactocentric)


In [49]:
import plotly.graph_objects as go

# Step 1: Create the scatter plot for the first line with both markers and lines
trace1 = go.Scatter3d(
    x=table['SkyCoord1_galactocentric'].x,  # x-axis (l_1)
    y=table['SkyCoord1_galactocentric'].y,  # y-axis (b_1)
    z=table['SkyCoord1_galactocentric'].z,  # z-axis (dist_1)
    mode='lines+markers',  # Show both markers and lines
    name='Star1',  # Legend name
    marker=dict(size=1),  # Size of the scatter points
    line=dict(width=1),  # Width of the line
    hovertext=table['time'],
    hoverinfo="text"
)

# Step 2: Create the scatter plot for the second line with both markers and lines
trace2 = go.Scatter3d(
    x=table['SkyCoord2_galactocentric'].x,  # x-axis (l_2)
    y=table['SkyCoord2_galactocentric'].y,  # y-axis (b_2)
    z=table['SkyCoord2_galactocentric'].z,  # z-axis (dist_2)
    mode='lines+markers',  # Show both markers and lines
    name='Star2',  # Legend name
    marker=dict(size=1),  # Size of the scatter points
    line=dict(width=1),  # Width of the line
    hovertext=table['time'],
    hoverinfo="text"
)

# Step 3: Combine the traces and create the layout
data = [trace1, trace2]  # Combine both lines

layout = go.Layout(
    title='3D Scatter Plot with Lines',  # Title of the plot
    scene=dict(
        xaxis_title='X (pc)',
        yaxis_title='Y (pc)',
        zaxis_title='Z (pc)',
    )
)

# Step 4: Create the figure and plot it
fig = go.Figure(data=data, layout=layout)
fig.show()
